In [1]:
%pwd

'c:\\Users\\seoju\\OneDrive\\문서\\workspace\\ESG 2024 prac'

### Yolo

In [ ]:
import yt_dlp
import cv2
from ultralytics import YOLO

In [12]:
def download_youtube_video(url, output_path):
    ydl_opts = {
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
    return filename

In [ ]:
def process_video(input_path, output_path):
    model = YOLO('yolov8l.pt')
    
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    for frame_count in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model(frame, conf=0.7)
        annotated_frame = results[0].plot()
        
        out.write(annotated_frame)
        
        if frame_count % fps == 10:  # 매 초마다 진행 상황 출력
            print(f"Processed {frame_count // fps} seconds")
    
    cap.release()
    out.release()
    print("Video processing completed")

youtube_url = "https://youtube.com/shorts/1vDoMuyk5ls?si=ZLQEjupeT-6vJesC"

input_video = download_youtube_video(youtube_url, "output")
output_video = "output/output_video_L.mp4"
process_video(input_video, output_video,) 

### Face Recognition

In [33]:
import face_recognition
import cv2
import numpy as np
import yt_dlp

def download_youtube_video(url, output_path):
    """YouTube 영상을 다운로드하는 함수"""
    ydl_opts = {
        'outtmpl': f'{output_path}/%(title)s.%(ext)s',
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        filename = ydl.prepare_filename(info)
    return filename

In [55]:
def process_fc_video(known_face_path, input_video_path, output_video_path, duration=60):
    known_image = face_recognition.load_image_file(known_face_path)
    known_face_encoding = face_recognition.face_encodings(known_image, num_jitters=10)[0]

    cap = cv2.VideoCapture(input_video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_count = 0
    max_frames = duration * fps if duration else float('inf')

    # 얼굴 감지 결과를 저장할 변수
    face_data = []
    detection_interval = 15  # 15프레임마다 얼굴 감지
    retention_frames = 45  # 감지된 얼굴 정보를 45프레임 동안 유지

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % detection_interval == 0:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            small_frame = cv2.resize(rgb_frame, (0, 0), fx=0.25, fy=0.25)

            try:
                face_locations = face_recognition.face_locations(small_frame)
                face_encodings = face_recognition.face_encodings(small_frame, face_locations, num_jitters=1)

                face_data = []
                for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                    matches = face_recognition.compare_faces([known_face_encoding], face_encoding, tolerance=0.4)
                    
                    name = "Unknown"
                    if True in matches:
                        name = "ccm"

                    face_data.append({
                        'location': (top * 4, right * 4, bottom * 4, left * 4),
                        'name': name
                    })
            except Exception as e:
                print(f"Error processing frame {frame_count}: {e}")

        # 저장된 얼굴 정보로 bbox와 label 그리기
        for face in face_data:
            top, right, bottom, left = face['location']
            name = face['name']

            # bbox 그리기 (두께 증가)
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 3)

            # label 배경 그리기 (반투명)
            overlay = frame.copy()
            cv2.rectangle(overlay, (left, bottom - 45), (right, bottom), (0, 0, 255), cv2.FILLED)
            cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)

            # label 텍스트 그리기 (크기와 두께 증가)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 2)

        out.write(frame)
        frame_count += 1

        if frame_count % fps == 0:
            print(f"Processed {frame_count // fps} seconds")

        # 일정 프레임이 지나면 face_data 초기화
        if frame_count % retention_frames == 0:
            face_data = []

    cap.release()
    out.release()
    print("Video processing completed")

In [56]:
youtube_url = "https://youtube.com/shorts/Ns9nsosC9Tc?si=2ZrPsKBp4Qc-oKZd"
known_face_path = "ccm.jpg"  # 침착맨의 얼굴 이미지 경로
input_video_path = download_youtube_video(youtube_url, "output")  # YouTube 비디오 다운로드
output_video_path = "output/face_output_ccm.mp4"  # 결과 비디오 파일 경로

process_fc_video(known_face_path, input_video_path, output_video_path)

[youtube] Extracting URL: https://youtube.com/shorts/Ns9nsosC9Tc?si=2ZrPsKBp4Qc-oKZd
[youtube] Ns9nsosC9Tc: Downloading webpage
[youtube] Ns9nsosC9Tc: Downloading ios player API JSON
[youtube] Ns9nsosC9Tc: Downloading m3u8 information
[info] Ns9nsosC9Tc: Downloading 1 format(s): 617+140
[download] output\진짜 웃긴 얘기.mp4 has already been downloaded
Processed 1 seconds
Processed 2 seconds
Processed 3 seconds
Processed 4 seconds
Processed 5 seconds
Processed 6 seconds
Processed 7 seconds
Processed 8 seconds
Processed 9 seconds
Processed 10 seconds
Processed 11 seconds
Processed 12 seconds
Processed 13 seconds
Processed 14 seconds
Processed 15 seconds
Processed 16 seconds
Processed 17 seconds
Processed 18 seconds
Processed 19 seconds
Processed 20 seconds
Processed 21 seconds
Processed 22 seconds
Processed 23 seconds
Processed 24 seconds
Processed 25 seconds
Processed 26 seconds
Processed 27 seconds
Processed 28 seconds
Processed 29 seconds
Processed 30 seconds
Processed 31 seconds
Processed 3

In [45]:
youtube_url = "https://youtube.com/shorts/jgvnWwsiUxA?si=PWiW5Z7wbG-4Ecx9"
known_face_path = "sj.jpg" 

In [53]:
input_video_path = download_youtube_video(youtube_url, "output")  # YouTube 비디오 다운로드
output_video_path = "output/face_output.mp4"  # 결과 비디오 파일 경로

process_fc_video(known_face_path, input_video_path, output_video_path)

[youtube] Extracting URL: https://youtube.com/shorts/jgvnWwsiUxA?si=PWiW5Z7wbG-4Ecx9
[youtube] jgvnWwsiUxA: Downloading webpage
[youtube] jgvnWwsiUxA: Downloading ios player API JSON
[youtube] jgvnWwsiUxA: Downloading m3u8 information
[info] jgvnWwsiUxA: Downloading 1 format(s): 617+140
[download] output\서로 하지말라는거 하는 한지민 이서진 ㅋㅋ.mp4 has already been downloaded
Processed 1 seconds
Processed 2 seconds
Processed 3 seconds
Processed 4 seconds
Processed 5 seconds
Processed 6 seconds
Processed 7 seconds
Processed 8 seconds
Processed 9 seconds
Processed 10 seconds
Processed 11 seconds
Processed 12 seconds
Processed 13 seconds
Processed 14 seconds
Processed 15 seconds
Processed 16 seconds
Processed 17 seconds
Processed 18 seconds
Processed 19 seconds
Processed 20 seconds
Processed 21 seconds
Processed 22 seconds
Processed 23 seconds
Processed 24 seconds
Processed 25 seconds
Processed 26 seconds
Processed 27 seconds
Processed 28 seconds
Processed 29 seconds
Processed 30 seconds
Processed 31 seco